# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lagoa,39.0500,-27.9833,66.87,85,13,21.94,PT,1730327880
1,1,rehoboth,-23.3167,17.0833,63.79,24,0,2.37,NaN,1730327881
2,2,port mathurin,-19.6833,63.4167,73.56,67,82,16.55,MU,1730327821
3,3,lihue,21.9789,-159.3672,81.99,71,75,20.71,US,1730327883
4,4,avarua,-21.2078,-159.7750,78.85,69,93,16.11,CK,1730327884


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [26]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 700,
    size = "Humidity",
    alpha=0.5,
    scale = 1,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [ ]:
# Narrow down cities that fit criteria and drop any results with null values


#cities with "Max Temp" equal or greater than 70, "Wind Speed" less than or equal to 8 and "Cloudiness" equal to 0
# Drop any rows with null values
cities_narrowed_down_df = city_data_df.loc[(city_data_df["Max Temp"] >= 70)  \
                                    & (city_data_df["Wind Speed"] <= 8) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()


# Display sample data
cities_narrowed_down_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
36,36,fort bragg,35.1390,-79.0060,78.64,62,0,4.61,US,1730327920
96,96,nelson bay,-32.7167,152.1500,76.10,60,0,1.99,AU,1730327986
183,183,mermoz boabab,14.7065,-17.4758,80.38,89,0,6.91,SN,1730328086
190,190,port said,31.2565,32.2841,71.58,60,0,6.91,EG,1730328094
214,214,salalah,17.0151,54.0924,78.89,57,0,4.61,OM,1730328125
255,255,virginia,37.5481,-77.4467,74.10,74,0,4.61,US,1730328174
257,257,pasni,25.2631,63.4710,78.78,70,0,6.82,PK,1730328176
277,277,remire-montjoly,4.9167,-52.2667,84.24,79,0,5.75,GF,1730328197


### Step 3: Create a new DataFrame called `hotel_df`.

In [49]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cities_narrowed_down_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()



# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
36,fort bragg,US,35.1390,-79.0060,62,
96,nelson bay,AU,-32.7167,152.1500,60,
183,mermoz boabab,SN,14.7065,-17.4758,89,
190,port said,EG,31.2565,32.2841,60,
214,salalah,OM,17.0151,54.0924,57,
255,virginia,US,37.5481,-77.4467,74,
257,pasni,PK,25.2631,63.4710,70,
277,remire-montjoly,GF,4.9167,-52.2667,79,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [55]:
##checking to make sure i have a connecton and check the url 
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit = 20
latitude = 35.1390	
longitude = 79.0060

params = {
    "categories":categories,
        "limit":limit,
        "apiKey":geoapify_key,
        "bias": f"proximity:{longitude},{latitude}",  # Geoapify uses "bias" for location
        "radius": radius}



    # Set base URL
base_url = "https://api.geoapify.com/v2/places"
    

    # Make and API request using the params dictionary
name_address = requests.get(base_url, params=params)
print(requests.get(base_url, params=params))
# print(name_address.status_code)  
print(name_address.url)

<Response [200]>
https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=20&apiKey=9211b133017b4b8abe8892a4dafc0803&bias=proximity%3A79.006%2C35.139&radius=10000


In [56]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit = 50

params = {
    "categories":categories,
        "limit":limit,
        "apiKey":geoapify_key
}
#    

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

          
    # Convert the API response to JSON format
    name_address = name_address.json()
    
          
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
fort bragg - nearest hotel: Airborne Inn Lodging
nelson bay - nearest hotel: No hotel found
mermoz boabab - nearest hotel: La maison blanche
port said - nearest hotel: The New Continental Hotel
salalah - nearest hotel: Muscat International Hotel
virginia - nearest hotel: Quirk Hotel
pasni - nearest hotel: Juddi Hotel
remire-montjoly - nearest hotel: Complexe Belova


,City,Country,Lat,Lng,Humidity,Hotel Name
36,fort bragg,US,35.1390,-79.0060,62,Airborne Inn Lodging
96,nelson bay,AU,-32.7167,152.1500,60,No hotel found
183,mermoz boabab,SN,14.7065,-17.4758,89,La maison blanche
190,port said,EG,31.2565,32.2841,60,The New Continental Hotel
214,salalah,OM,17.0151,54.0924,57,Muscat International Hotel
255,virginia,US,37.5481,-77.4467,74,Quirk Hotel
257,pasni,PK,25.2631,63.4710,70,Juddi Hotel
277,remire-montjoly,GF,4.9167,-52.2667,79,Complexe Belova


In [57]:
# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
36,fort bragg,US,35.1390,-79.0060,62,Airborne Inn Lodging
96,nelson bay,AU,-32.7167,152.1500,60,No hotel found
183,mermoz boabab,SN,14.7065,-17.4758,89,La maison blanche
190,port said,EG,31.2565,32.2841,60,The New Continental Hotel
214,salalah,OM,17.0151,54.0924,57,Muscat International Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [60]:
%%capture --no-display

# Configure the map plot
map_plot_1 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 700,
    size = "Humidity",
    scale = 2,
    alpha=0.5,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
     #hover_cols = ["Country", "Hotel Name"]
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)